# Metrics
https://github.com/hsiangyuzhao/Segmentation-Metrics-PyTorch/blob/master/metric.py

In [ ]:
# unit test
import unittest

import glob

import torch
from torch.utils.data import DataLoader

import segmentation_models_pytorch as smp

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, r"helper")
sys.path.insert(0, r"/helper")
sys.path.insert(0, r"./helper")
sys.path.insert(0, r"../helper")
print(sys.path)


from dataset.meanteacher import *

# relevant: import the UncertaintyMetric
from compute.metric.segmentation import * 
from compute.metric.symmetric_hausdorff import * 

In [ ]:
# preparation
channels = 1
n_output_neurons=2
model = smp.UnetPlusPlus(
                    encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
                    in_channels=channels,          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                    classes=n_output_neurons,         # model output channels (number of classes in your dataset)
                )
csv_filenames = glob.glob(r"../data_prep/mt_*.csv")
# image size has to be dividable by 32!!
dataset = MeanTeacherTrainDataset(mode="train", channels=channels, image_size=512, csv_filenames=csv_filenames)
train_loader = DataLoader(dataset, batch_size=5, shuffle=True)

# relevant: create instance of class
s = SegmentationMetrics()
symhd = SymmetricHausdorffMetric()

model_output = None
# load one batch

model_output_area = []
ground_truth_area = []

for idx, item in enumerate(train_loader):
    
    print("next batch")
    
    model.eval()
    with torch.no_grad():
        model_output = model(item["img"])

        
        # item["msk"][0].permute(1, 2, 0).detach().cpu().numpy()
        model_output_noise = model_output * item["msk"]
        
        for msk, pred in zip(item["msk"], model_output_noise):
            
            print("pred", pred.shape)
            print("msk", msk.shape)
            
            _, highest_class = torch.max(pred, 1)  
            
            print("high", highest_class.shape)

            model_output_area.append(torch.sum(highest_class))

            ground_truth_area.append(torch.sum(msk))

    

    if idx == 0:
        break




In [ ]:
# model_output_noise = model_output * item["msk"]

#print(model_output)

from scipy.spatial.distance import directed_hausdorff
    
print(s(y_true=item["msk"], y_pred=model_output_noise))
print(symhd(model_output_noise, item["msk"]))

In [ ]:
plt.figure()
plt.imshow( item["msk"][0].permute(1, 2, 0).detach().cpu().numpy(), cmap="gray" )



In [ ]:
print(model_output.shape)

_, highest_class = torch.max(model_output_noise, 1)   
# highest_class = highest_class.detach().cpu().numpy()

print(highest_class.shape)

#redo = highest_class.permute(1, 2, 0)
print(highest_class[0].shape)

plt.figure()
plt.imshow(highest_class[0], cmap="gray")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm


max_num = max(max(model_output_area), max(ground_truth_area))

c = [abs(ele1 - ele2) for ele1, ele2 in zip(model_output_area, ground_truth_area)]

plt.figure()
plt.xlabel("predicted area")
plt.ylabel("ground truth area")
plt.scatter(model_output_area, ground_truth_area, c=c, cmap='cool')
plt.plot([0, max_num], [0, max_num], color = 'k')

plt.savefig("example_results/area.png", dpi=1200)

